In [3]:
import bisect
import numpy
from numpy.linalg import norm
from numpy.linalg import svd 
from scipy.sparse.linalg import svds

In [7]:
def pcp_alm(X, maxiter=500, tol=1e-7):
    """Principal Component Pursuit
    Finds the Principal Component Pursuit solution.
    Solves the optimization problem::
        (L^*,S^*) = argmin || L ||_* + gamma * || S ||_1
                    (L,S)
                    subject to    L + S = X
    where || . ||_* is the nuclear norm.  Uses an augmented Lagrangian approach
    Parameters
    ----------
    X : array of shape [n_samples, n_features]
        Data matrix.
    maxiter : int, 500 by default
        Maximum number of iterations to perform.
    tol : float, 1e-7 by default - in the paper

    Returns
    -------
    L : array of shape [n_samples, n_features]
        The low rank component
    S : array of shape [n_samples, n_features]
        The sparse component
    (u, sig, vt) : tuple of arrays
        SVD of L.
    n_iter : int
        Number of iterations
    Reference
    ---------
       Candes, Li, Ma, and Wright
       Robust Principal Component Analysis?
       Submitted for publication, December 2009.
    """
    
    
    def soft_threshold(v = 0, tau = 0):
        '''
        shrinkiage opterator S_tau[x]
        '''
        tmp = numpy.abs(v)
        tmp = numpy.subtract(tmp, tau)
        tmp = numpy.maximum(tmp, 0.0)
        return numpy.multiply(numpy.sign(v), tmp)
    
    def svt(X, tau, k, svd_function, out=None):
        def svd_reconstruct(u, sig, v, out=None, tmp_out=None):
            tmp = np.multiply(u, sig, out=tmp_out)
            return np.dot(tmp, v, out=out)

        def truncate_top_k_svd(u, sig, v, k):
            return u[:, :k], sig[:k], v[:k, :]

        m, n = X.shape
        
        u, sig, v = svd_function(X, k)

        sig = soft_threshold(sig, tau)
        r = numpy.sum(sig > 0)
        
        u, sig, v = svd_function(X, r)
        sig = soft_threshold(sig, tau)

        if r > 0:
            print("Z= reconstructed")
            u, sig, v = truncate_top_k_svd(u, sig, v, r)
            print("reconstructed sig =", sig)
            Z = svd_reconstruct(u, sig, v, out=out)
        else:
            print("Z= 0")
            out[:] = 0
            Z = out
            u, sig, v = numpy.empty((m, 0)), numpy.empty((0, 0)), numpy.empty((0, n))
        return (Z, r, (u, sig, v))
    
    def svd_choice(n, d):
        '''
        choose svd depend on the size 

        return 'dense_top_k_svd/sparse_svds
        '''
    
        ratio = float(d) / float(n)
        vals = [(0, 0.02), (100, 0.06), (200, 0.26),
                (300, 0.28), (400, 0.34), (500, 0.38)]

        i = bisect.bisect_left([r[0] for r in vals], n)
        choice = dense_top_k_svd if ratio > vals[i-1][1] else svds
        return choice
    
    def dense_top_k_svd(A, k):
        '''
        A - matrix
        k - Top K components
        '''
        u, sig, v = svd(A, full_matrices=0)
        return u[:, :k], sig[:k], v[:k, :]
    
    n = X.shape
    frob_norm = numpy.linalg.norm(X, 'fro')
    two_norm = numpy.linalg.norm(X, 2)
    one_norm = numpy.sum(numpy.abs(X))
    inf_norm = numpy.max(numpy.abs(X))
    
    print("frob_norm", frob_norm)
    print("two_norm", two_norm)
    print("one_norm", one_norm)
    print("info_norm", inf_norm)

    mu_inv = 4 * one_norm / numpy.prod(n)
    
    gamma = 1/numpy.sqrt(numpy.max([X.shape[0], X.shape[1]]))
    print("gamma", gamma)
    # Kicking
    k = numpy.min([
        numpy.floor(mu_inv / two_norm),
        numpy.floor(gamma * mu_inv / inf_norm)
    ])
    Y = k * X
    sv = 10

    # Variable init
    S = numpy.zeros(n)
    
    print("k",k)
    print("mu_inv", mu_inv)
    
    for i in range(maxiter):
        
        # Shrink singular values
        l= X - S + mu_inv*Y
        svd_fun = svd_choice(numpy.min(l.shape), sv)
        print(svd_fun)
        print("sv", sv)
        L, r, (u, sig, v) = svt(l, mu_inv, sv, svd_function = svd_fun)
        print("non-zero sigular value", r)          
        if r < sv:
            sv = numpy.min([r + 1, numpy.min(n)])
        else:
            sv = numpy.min([r +  int(numpy.round(0.05 * numpy.min(n))), numpy.min(n)])
        
        # Shrink entries
        s = X - L + mu_inv*Y
        S = soft_threshold(s, gamma * mu_inv)
    
        # Check convergence
        R= X - L - S
        stopCriterion = numpy.linalg.norm(R, 'fro') / frob_norm
        print("stopCriterion", stopCriterion)
        if stopCriterion < tol:
            break

        # Update dual variable
        Y += 1/mu_inv *(X - L - S) 

    return L, S, (u, sig, v), i + 1

In [8]:
###test###

import numpy as np

n = 100
r = 3
np.random.seed(123)
base = 100 + np.cumsum(np.random.randn(n, r), axis=0)
scales = np.abs(np.random.randn(n, r))
L = np.dot(base, scales.T)
S = np.round(0.25 * np.random.randn(n, n))
M = L + S

L_hat, S_hat, r, niter = pcp_alm(M,500)
print(np.max(np.abs(S - S_hat)))
print(np.max(np.abs(L - L_hat)))
print(niter)



_, s, _ = np.linalg.svd(L, full_matrices=False)
print (s[s > 1e-11])

_, s_hat, _ = np.linalg.svd(L_hat, full_matrices=False)
print (s_hat[s_hat > 1e-11])

frob_norm 25592.2494664
two_norm 25589.5984847
one_norm 2355268.6747
info_norm 526.610206522
gamma 0.1
k 0.0
mu_inv 942.107469879
<function pcp_alm.<locals>.dense_top_k_svd at 0x10f467d90>
sv 10
Z= reconstructed
reconstructed sig = [ 24647.4910148]
non-zero sigular value 1
stopCriterion 0.0395259325301
<function svds at 0x10ed25048>
sv 2
Z= reconstructed
reconstructed sig = [ 25589.59848468]
non-zero sigular value 1
stopCriterion 0.014393051743
<function svds at 0x10ed25048>
sv 2
Z= reconstructed
reconstructed sig = [    80.91406357  25589.59848468]
non-zero sigular value 2
stopCriterion 0.0115282303434
<function pcp_alm.<locals>.dense_top_k_svd at 0x10f467d90>
sv 7
Z= reconstructed
reconstructed sig = [ 25589.59848468    341.00717782]
non-zero sigular value 2
stopCriterion 0.00544189803452
<function pcp_alm.<locals>.dense_top_k_svd at 0x10f467d90>
sv 3
Z= reconstructed
reconstructed sig = [ 25589.59848468    341.00717782]
non-zero sigular value 2
stopCriterion 0.00544189803452
<functi

#### Reference
Code Reference: https://github.com/apapanico/sklearn-rpca/blob/master/sklearnrpca/rpca_alm.py 
Modified based on the alternativate direction algorithm presented in the original paper: https://statweb.stanford.edu/~candes/papers/RobustPCA.pdf